# Music Store Sales Data Analysis

This notebook runs an end-to-end analysis on a sample music store dataset. It includes:

- Instructions to prepare the SQLite database
- Exploratory SQL queries
- Python (pandas) analysis: RFM, clustering, charts

**Files included in the project:** `create_db.sql`, CSV files, this notebook, and `music_store.db` (created below).

In [ ]:
# Setup - imports
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
%matplotlib inline
print("Libraries loaded")

In [ ]:
# Connect to the SQLite database (music_store.db must be in the same folder)
conn = sqlite3.connect('music_store.db')
print("Connected to music_store.db")

In [ ]:
# Load tables into pandas
customers = pd.read_sql_query("SELECT * FROM customers", conn)
artists = pd.read_sql_query("SELECT * FROM artists", conn)
genres = pd.read_sql_query("SELECT * FROM genres", conn)
albums = pd.read_sql_query("SELECT * FROM albums", conn)
tracks = pd.read_sql_query("SELECT * FROM tracks", conn)
invoices = pd.read_sql_query("SELECT * FROM invoices", conn)
invoice_items = pd.read_sql_query("SELECT * FROM invoice_items", conn)

print(customers.shape, tracks.shape, invoices.shape, invoice_items.shape)
customers.head()

In [ ]:
# Quick validation: ensure invoice totals match sum of invoice_items per invoice
agg = invoice_items.groupby('invoice_id')['line_total'].sum().reset_index().rename(columns={'line_total':'sum_line_total'})
check = invoices.merge(agg, on='invoice_id', how='left')
check['diff'] = check['total'] - check['sum_line_total']
print(check[['invoice_id','total','sum_line_total','diff']])

In [ ]:
# Top genres by revenue
ii = invoice_items.merge(tracks[['track_id','genre_id']], on='track_id', how='left')
ii = ii.merge(genres, on='genre_id', how='left')
genre_revenue = ii.groupby('name', as_index=False)['line_total'].sum().sort_values('line_total', ascending=False)
genre_revenue

In [ ]:
# Plot genre revenue
plt.figure(figsize=(8,4))
plt.bar(genre_revenue['name'], genre_revenue['line_total'])
plt.title('Revenue by Genre')
plt.xlabel('Genre')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# RFM calculation
invoices['invoice_date'] = pd.to_datetime(invoices['invoice_date'])
snapshot_date = pd.to_datetime('2025-03-01')

rfm = invoices.groupby('customer_id').agg({
    'invoice_date': lambda x: (snapshot_date - x.max()).days,
    'invoice_id': 'count',
    'total': 'sum'
}).reset_index()

rfm.columns = ['customer_id','recency_days','frequency','monetary']
rfm = rfm.merge(customers[['customer_id','first_name','last_name']], on='customer_id', how='left')
rfm['customer_name'] = rfm['first_name'] + ' ' + rfm['last_name']
rfm

In [ ]:
# RFM scoring (quartiles)
rfm['r_quartile'] = pd.qcut(rfm['recency_days'].rank(method='first'), 4, labels=[4,3,2,1]).astype(int)
rfm['f_quartile'] = pd.qcut(rfm['frequency'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
rfm['m_quartile'] = pd.qcut(rfm['monetary'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
rfm['RFM_Score'] = rfm['r_quartile'].astype(str) + rfm['f_quartile'].astype(str) + rfm['m_quartile'].astype(str)
rfm

In [ ]:
# KMeans clustering on RFM
X = rfm[['recency_days','frequency','monetary']].fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm['cluster'] = kmeans.fit_predict(X_scaled)
rfm.groupby('cluster').agg({'recency_days':'mean','frequency':'mean','monetary':'mean','customer_id':'count'}).reset_index()

In [ ]:
# Monthly revenue time series
invoices['ym'] = invoices['invoice_date'].dt.to_period('M').astype(str)
monthly = invoices.groupby('ym', as_index=False)['total'].sum().sort_values('ym')
monthly

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(monthly['ym'], monthly['total'], marker='o')
plt.title('Monthly Revenue')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()